In [1]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from math import pi,e
from numpy import linspace

In [2]:
mh = 0.1
order = 1
endT = 1
mesh = Mesh(unit_square.GenerateMesh(maxh=mh))

In [3]:
# penalty parameters
beta_u = 100 
beta_p = 100
# stablization paramer
gamma_p = 1000

# physical parameters
mu  = 1
lam = 1
c0 = 1e-4
alpha = 1
K = 1

def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(2)

In [4]:
# 定义解析解
# t = CF(z)
t = Parameter(0.0) # A Parameter is a constant CoefficientFunction the value of which can be changed with the Set-function.
u_x = e**(-t)*(sin(2*pi*y)*(-1+cos(2*pi*x))+sin(pi*x)*sin(pi*y)/(lam+mu))
u_y = e**(-t)*(sin(2*pi*x)*(1-cos(2*pi*y))+sin(pi*x)*sin(pi*y)/(lam+mu))
exact_u = CF((u_x,u_y))
exact_p = e**(-t)*sin(pi*x)*sin(pi*y)

# strain tensor
epsilon_xx = u_x.Diff(x)
epsilon_yy = u_y.Diff(y) 
epsilon_xy = 0.5*(u_x.Diff(y) +  u_y.Diff(x))

# total stress tensor
sigma_xx = lam*(epsilon_xx + epsilon_yy) + 2*mu*epsilon_xx - alpha*exact_p
sigma_yy = lam*(epsilon_xx + epsilon_yy) + 2*mu*epsilon_yy - alpha*exact_p
sigma_xy = 2*mu*epsilon_xy

# 右端项 f_x, f_y
f_x = - (sigma_xx.Diff(x) + sigma_xy.Diff(y))
f_y = - (sigma_xy.Diff(x) + sigma_yy.Diff(y))

# 向量形式
F = CF( (f_x,f_y) )
g = (c0*exact_p+alpha*(u_x.Diff(x)+u_y.Diff(y))).Diff(t) - K*(exact_p.Diff(x).Diff(x)+exact_p.Diff(y).Diff(y))

uD = exact_u
pD = exact_p

In [5]:
# DG spaces
U = VectorL2(mesh, order=order, dirichlet=".*", dgjumps=True) # space for velocity
P = L2(mesh, order=order, dirichlet=".*", dgjumps=True) # space for pressure
fes = U*P
(u,p), (v,q) = fes.TnT()
gfu = GridFunction(fes)

# Define the jumps and the averages
jump_u = u - u.Other()
jump_v = v - v.Other()
jump_p = p - p.Other()
jump_q = q - q.Other()
n = specialcf.normal(2)
strain_u = Sym(Grad(u))
strain_v = Sym(Grad(v))
mean_stress_u = 0.5*(Stress(Sym(Grad(u)))+Stress(Sym(Grad(u.Other()))))*n
mean_stress_v = 0.5*(Stress(Sym(Grad(v)))+Stress(Sym(Grad(v.Other()))))*n
mean_dpdn = 0.5*K*(grad(p)+grad(p.Other()))*n
mean_dqdn = 0.5*K*(grad(q)+grad(q.Other()))*n
mean_p = 0.5*(p + p.Other())
mean_q = 0.5*(q + q.Other())
h = specialcf.mesh_size   

In [6]:
ah = BilinearForm(fes)
# Au
ah += 2*mu*InnerProduct(strain_u,strain_v)*dx + lam*div(u)*div(v)*dx \
        - (InnerProduct(mean_stress_u,jump_v) + InnerProduct(mean_stress_v,jump_u) - beta_u/h*InnerProduct(jump_u,jump_v))*dx(skeleton=True) \
        - (InnerProduct(Stress(Sym(Grad(u)))*n,v) + InnerProduct(Stress(Sym(Grad(v)))*n,u) - beta_u/h*InnerProduct(u,v))*ds(skeleton=True)
# -B
ah += -alpha*(div(v)*p*dx - mean_p*jump_v*n*dx(skeleton=True) - p*v*n*ds(skeleton=True))
# Ap
ah += K*grad(p)*grad(q)*dx \
        - (mean_dpdn*jump_q + mean_dqdn*jump_p - beta_p/h*jump_p*jump_q)*dx(skeleton=True) \
        - (K*grad(p)*n*q + K*grad(q)*n*p - beta_p/h*p*q)*ds(skeleton=True) 
ah.Assemble()

mh = BilinearForm(fes)
# C
mh += c0*p*q*dx + gamma_p*h*h*grad(p)*grad(q)*dx
# B^T
mh += alpha*(div(u)*q*dx - mean_q*jump_u*n*dx(skeleton=True) - q*u*n*ds(skeleton=True))
mh.Assemble()

In [7]:
dt = 0.001
mstar = mh.mat.CreateMatrix()
# corresponds to M* = M/dt + A
mstar.AsVector().data = 1/dt*mh.mat.AsVector() + ah.mat.AsVector()
invmstar = mstar.Inverse(freedofs=fes.FreeDofs())

In [8]:
# r.h.s
f = LinearForm(fes)
f += F*v*dx - InnerProduct(uD,Stress(Sym(Grad(v)))*n)*ds(skeleton=True) + beta_u/h*uD*v*ds(skeleton=True)
f += g*q*dx - alpha*q*uD*n*ds(skeleton=True) - K*grad(q)*n*pD*ds(skeleton=True) + beta_p/h*pD*q*ds(skeleton=True)

In [9]:
gfu = GridFunction(fes)
def TimeStepping(invmstar, initial_condu = None, initial_condp = None, t0 = 0, tend = 1,
                      nsamples = 10):
    if initial_condu and initial_condp :
        gfu.components[0].Set(initial_condu)
        gfu.components[1].Set(initial_condp)
    cnt = 0; # 时间步计数
    time = t0 # 当前时间
    sample_int = int(floor(tend / dt / nsamples)+1) # 采样间隔，用于决定什么时候把解存入 gfut
    gfut = GridFunction(gfu.space,multidim=0) #存储所有采样时间步的结果，多维 GridFunction
    gfut.AddMultiDimComponent(gfu.vec)
    while time <  tend + 1e-7:
        t.Set(time)
        f.Assemble()
        res = f.vec + 1/dt * mh.mat * gfu.vec
        gfu.vec.data = invmstar * res
        print("\r",time,end="")
        # print(time,end="\n")
        if cnt % sample_int == 0:
            gfut.AddMultiDimComponent(gfu.vec)
        cnt += 1; time = cnt * dt
    return gfut,gfu

In [10]:
gfut,gfu = TimeStepping(invmstar, initial_condu=exact_u, initial_condp=exact_p, tend=endT, nsamples=20)
Draw(gfut.components[0], mesh, interpolate_multidim=True, animate=True, autoscale=False)
Draw(gfut.components[1], mesh, interpolate_multidim=True, animate=True, autoscale=False)

 1.099000000000000144

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [12]:
gff = GridFunction(fes)
gfft = GridFunction(gff.space,multidim=0)
num_frames = 21 
for time_point in linspace(0, endT, num_frames):
    t.Set(time_point)
    gff.components[0].Set(exact_u)
    gff.components[1].Set(exact_p)
    gfft.AddMultiDimComponent(gff.vec)
Draw(gfft.components[0], mesh, interpolate_multidim=True, animate=True, autoscale=False)
Draw(gfft.components[1], mesh, interpolate_multidim=True, animate=True, autoscale=False)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [11]:
error_u = sqrt(Integrate((gfu.components[0] - exact_u)**2, mesh))
print(error_u)
error_p = sqrt(Integrate((gfu.components[1] - exact_p)**2, mesh))
print(error_p)

0.027168473824172475
0.2845131600641728
